In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install emoji

     |████████████████████████████████| 174 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=ee12dbd7ee623465836bd3ecb7374c90ce8648b4bbbd9504946f8fdf39cc413e
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [3]:
#download
project_path = "/content/drive/MyDrive/DNLP/hate-speech_BERT_final/"
davidson_path = project_path + "datasets/davidson/davidson.csv"
!wget "https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv" $project_path"davidson"
!mv labeled_data.csv $davidson_path

HATE = 0
OFFENSIVE = 1
NEITHER = 2

--2022-02-15 15:50:51--  https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2546446 (2.4M) [text/plain]
Saving to: ‘labeled_data.csv’

labeled_data.csv    100%[===================>]   2.43M  --.-KB/s    in 0.07s   

2022-02-15 15:50:51 (36.5 MB/s) - ‘labeled_data.csv’ saved [2546446/2546446]

/content/drive/MyDrive/DNLP/hate-speech_BERT_final/davidson: Scheme missing.
FINISHED --2022-02-15 15:50:51--
Total wall clock time: 0.2s
Downloaded: 1 files, 2.4M in 0.07s (36.5 MB/s)


In [4]:
import pandas as pd
davidson = pd.read_csv(davidson_path, usecols=["class", "tweet"])

In [5]:
davidson

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [6]:
import re
import numpy as np
import emoji as emoji
import string
import pandas as pd

In [7]:
def read_dataset(ds):
    #data = pd.read_csv("labeled_data.csv")
    #data = data.drop(['count', 'hate_speech', 'offensive_language', 'neither'], axis=1)
    return ds['tweet'].tolist(), data['class']


def pre_process_dataset(values):
    new_values = list()

    emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '</3', ':\*',
                 ';-)',
                 ';)', ';-D', ';D', '(;', '(-;', ':-(', ':(', '(:', '(-:', ':,(', ':\'(', ':"(', ':((', ':D', '=D',
                 '=)',
                 '(=', '=(', ')=', '=-O', 'O-=', ':o', 'o:', 'O:', 'O:', ':-o', 'o-:', ':P', ':p', ':S', ':s', ':@',
                 ':>',
                 ':<', '^_^', '^.^', '>.>', 'T_T', 'T-T', '-.-', '*.*', '~.~', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp',
                 ':-|',
                 ':->', ':-<', '$_$', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X', '<','>']

    for value in values:
        text = value.lower()
        for emo in text.split():
            if emo in emoji.UNICODE_EMOJI:
                text = text.replace(emo, "<emoticon>")
            if emo in emoticons:
                text = text.replace(emo, "<emoticon>")


        users = re.findall("[@]\w+", text)
        #text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)

        for user in users:
            text = text.replace(user, "<user>")
        urls = re.findall(r'(https?://[^\s]+)', text)
        if len(urls) != 0:
            for url in urls:
                text = text.replace(url, "<url >")
        
        numbers = re.findall('[0-9]+', text)
        for number in numbers:
            text = text.replace(number, "<number>")
        text = text.replace('#', "<hashtag>")
        symbols="!@#$%^&*()[]{};:,./?\|`~-=_+\""
        for symbol in symbols:
            text=text.replace(symbol, " ")
        text = re.sub("(rt)", "", text)
        text = re.sub("\s+", " ", text)

        #text = "".join(l for l in text if l not in string.punctuation)
        #text = re.sub(r'[" "]+', " ", text)
        new_values.append(text)
    return new_values


def data_process(data, labels):
    input_ids = []
    attention_masks = []
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    for sentence in data:
        bert_inp = bert_tokenizer.__call__(sentence, max_length=36,
                                           padding='max_length', pad_to_max_length=True,
                                           truncation=True, return_token_type_ids=False)

        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])
    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    labels = np.array(labels)
    return input_ids, attention_masks, labels


def load_and_process():
    data, labels = read_dataset()
    num_of_labels = len(labels.unique())
    input_ids, attention_masks, labels = data_process(pre_process_dataset(data), labels)

    return input_ids, attention_masks, labels

In [8]:
preprocessed_data = pre_process_dataset(davidson["tweet"].tolist())
preprocessed_data

[" <user> as a woman you shouldn't complain about cleaning up your house amp as a man you should always take the trash out ",
 ' <user> boy dats cold tyga dwn bad for cuffin dat hoe in the <number>st place ',
 ' <user> dawg <user> you ever fuck a bitch and she sta to cry you be confused as shit',
 ' <user> <user> she look like a tranny',
 ' <user> the sh<emoticon> you hear about me might be true or <emoticon> might be faker than the b<emoticon>ch who told <emoticon> to ya <hashtag><number> ',
 ' <user> the shit just blows me claim you so faithful and down for somebody but still fucking with hoes <hashtag><number> <hashtag><number> <hashtag><number> ',
 ' <user> i can not just sit up and hate on another bitch i got too much shit going on ',
 " <hashtag><number> <user> cause i'm tired of you big bitches coming for us skinny girls <hashtag><number> ",
 ' amp you might not get ya bitch back amp thats that ',
 ' <user> hobbies include fighting mariam bitch',
 ' keeks is a bitch she curves e

In [9]:
valid_words = set()
with open(project_path+"utilities/word_list.txt") as f:
  for line in f:
    valid_words.add(line.rstrip())


In [10]:
def remove_duplicate_characters(text):
    repeat_pattern = re.compile(r'(\w)\1*')
    text= repeat_pattern.sub(r'\1',text)
    return text

In [11]:
#elongated 
pre2 = []
for tweet in preprocessed_data:
  substitute_tweet = ""
  for word in tweet.split():
    substitution = word
    if word not in valid_words:
      #remove elongations
      candidate = remove_duplicate_characters(word)
      #print(candidate)
      if candidate in valid_words:
        substitution = candidate
    substitute_tweet = substitute_tweet + substitution + " "
  pre2.append(substitute_tweet)

In [12]:
pre2

["<user> as a woman you shouldn't complain about cleaning up your house amp as a man you should always take the trash out ",
 '<user> boy dats cold tyga dwn bad for cuffin dat hoe in the <number>st place ',
 '<user> dawg <user> you ever fuck a bitch and she sta to cry you be confused as shit ',
 '<user> <user> she look like a tranny ',
 '<user> the sh<emoticon> you hear about me might be true or <emoticon> might be faker than the b<emoticon>ch who told <emoticon> to ya <hashtag><number> ',
 '<user> the shit just blows me claim you so faithful and down for somebody but still fucking with hoes <hashtag><number> <hashtag><number> <hashtag><number> ',
 '<user> i can not just sit up and hate on another bitch i got too much shit going on ',
 "<hashtag><number> <user> cause i'm tired of you big bitches coming for us skinny girls <hashtag><number> ",
 'amp you might not get ya bitch back amp thats that ',
 '<user> hobbies include fighting mariam bitch ',
 'keeks is a bitch she curves everyone 

In [13]:
from math import log

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
words = open(project_path+"utilities/word_list2.txt").read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [14]:
infer_spaces("<hashtag>redapple")

'<hashtag> red apple'

In [15]:
pre3 = list()
for tweet in pre2:
  rebuilt = list()
  tokens = tweet.split()
  for token in tokens:
    if token.startswith("<hashtag>"):
      rebuilt.append(infer_spaces(token))
    else:
      rebuilt.append(token)
  recomposed = listToStr = ' '.join(map(str, rebuilt))
  pre3.append(recomposed)

In [16]:
pre3

["<user> as a woman you shouldn't complain about cleaning up your house amp as a man you should always take the trash out",
 '<user> boy dats cold tyga dwn bad for cuffin dat hoe in the <number>st place',
 '<user> dawg <user> you ever fuck a bitch and she sta to cry you be confused as shit',
 '<user> <user> she look like a tranny',
 '<user> the sh<emoticon> you hear about me might be true or <emoticon> might be faker than the b<emoticon>ch who told <emoticon> to ya <hashtag> <number>',
 '<user> the shit just blows me claim you so faithful and down for somebody but still fucking with hoes <hashtag> <number> <hashtag> <number> <hashtag> <number>',
 '<user> i can not just sit up and hate on another bitch i got too much shit going on',
 "<hashtag> <number> <user> cause i'm tired of you big bitches coming for us skinny girls <hashtag> <number>",
 'amp you might not get ya bitch back amp thats that',
 '<user> hobbies include fighting mariam bitch',
 'keeks is a bitch she curves everyone lol 

In [17]:
preprocessed_df = pd.DataFrame(pre3)
preprocessed_df

,0
0,<user> as a woman you shouldn't complain about...
1,<user> boy dats cold tyga dwn bad for cuffin d...
2,<user> dawg <user> you ever fuck a bitch and s...
3,<user> <user> she look like a tranny
4,<user> the sh<emoticon> you hear about me migh...
...,...
24778,you's a muthaf in lie <hashtag> <number> <user...
24779,you've gone and broke the wrong hea baby and d...
24780,young buck wanna eat dat nigguh like i aint fu...
24781,you got wild bitches tellin you lies


In [18]:
preprocessed_df["length"] = [len(t) for t in preprocessed_df[0]]

In [19]:
preprocessed_df = preprocessed_df[preprocessed_df["length"]>=2]

In [26]:
!pip install pytorch_pretrained_bert
!pip install folium==0.2.1
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 69 kB 3.3 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=660efad86088d5342b742588615de8f09c13a7df6db20df48837d642730b3921
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
davidson_pre = davidson.copy()
davidson_pre['Mapped_label'] = davidson_pre["class"]
davidson_pre["class"] = davidson_pre["class"].map({NEITHER:"neither", OFFENSIVE: "offensive", HATE: "hate"})
davidson_pre['ProcessedText'] = preprocessed_df[0]
davidson_pre['ProcessedText_BERT'] = ["[CLS] "+ t for t in davidson_pre["ProcessedText"]]
davidson_pre['ProcessedText_BERTbase_length'] = [len(tokenizer.tokenize(sent)) for sent in davidson_pre["ProcessedText_BERT"]]

In [ ]:
print("neither: ", end="")
print(len(davidson_pre[davidson_pre["class"] == "neither"]))
print("hate: ", end="")
print(len(davidson_pre[davidson_pre["class"] == "hate"]))
print("offensive: ", end="")
print(len(davidson_pre[davidson_pre["class"] == "offensive"]))
print(f"total: {len(davidson_pre)}")

In [ ]:
davidson_pre.to_csv(project_path+"datasets/davidson/davidson_preprocessed.csv")